In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, roc_auc_score, roc_curve, confusion_matrix, classification_report

In [ ]:
df=pd.read_csv('../input/pima-indians-diabetes-database/diabetes.csv')

In [ ]:
df.head()

In [ ]:
df.describe().T

In [ ]:
from pandas_profiling import ProfileReport 

profile = ProfileReport( df, title='Pandas profiling report ' , html={'style':{'full_width':True}})

profile.to_notebook_iframe()

In [ ]:
#Check Missing Values : 
df.isna().sum().plot(kind='bar')

In [ ]:
#Now let's look to the data : 
plt.style.use('ggplot') # Using ggplot2 style visuals 

f, ax = plt.subplots(figsize=(12, 15))

ax.set_facecolor('#fafafa')
ax.set(xlim=(-.05, 200))
plt.ylabel('Variables')
plt.title("Overview Data Set")
ax = sns.boxplot(data = df, 
  orient = 'h', 
  palette = 'Set2')

**From the above EDA :**

**we have the following observations :**


1-Columns ( Glucise , Blood Pressure , Skin Thickness , Insulin , BMI ) has no logical 0 values , specially skin thickness and insulin.

2-Pregnancies have some outliers

3-Target ( outcome) is unbalanced the number of non-diabetic is 268 the number of diabetic patients is 500

4- we observe also some skew in the data but we can deal with that later 


In [ ]:
#Now , replce 0 values with nan so we can impute the values : 
df[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']] = df[['Glucose','BloodPressure','SkinThickness','Insulin','BMI']].replace(0,np.NaN)

In [ ]:
#Check Nan values : 
#Check Missing Values : 
df.isna().sum().plot(kind='bar')

In [ ]:
#To best impute missing data , we will divide our data sets into two :
#Diabetes & NO Diabetes :
Diabetes = df[(df['Outcome'] != 0)]
No_diab = df[(df['Outcome'] == 0)]

In [ ]:
Diabetes['Glucose'].mean()

In [ ]:
Diabetes.hist(figsize=(15,12));

In [ ]:
No_diab.hist(figsize=(15,12));

In [ ]:
# #according to the data distribution :
# # we will impute missing values with median , according to Diabetes and no diabetes data sets : 

# df.loc[(df['Outcome'] == 0 ) & (df['Insulin'].isnull()), 'Insulin'] = No_diab['Insulin'].median()
# df.loc[(df['Outcome'] == 1 ) & (df['Insulin'].isnull()), 'Insulin'] = Diabetes['Insulin'].median()
# df.loc[(df['Outcome'] == 0 ) & (df['Glucose'].isnull()), 'Glucose'] = No_diab['Glucose'].median()
# df.loc[(df['Outcome'] == 1 ) & (df['Glucose'].isnull()), 'Glucose'] = Diabetes['Glucose'].median()
# df.loc[(df['Outcome'] == 0 ) & (df['SkinThickness'].isnull()), 'SkinThickness'] = No_diab['SkinThickness'].median()
# df.loc[(df['Outcome'] == 1 ) & (df['SkinThickness'].isnull()), 'SkinThickness'] = Diabetes['SkinThickness'].median()
# df.loc[(df['Outcome'] == 0 ) & (df['BloodPressure'].isnull()), 'BloodPressure'] = No_diab['BloodPressure'].median()
# df.loc[(df['Outcome'] == 1 ) & (df['BloodPressure'].isnull()), 'BloodPressure'] = Diabetes['BloodPressure'].median()
# df.loc[(df['Outcome'] == 0 ) & (df['BMI'].isnull()), 'BMI'] = No_diab['BMI'].median()
# df.loc[(df['Outcome'] == 1 ) & (df['BMI'].isnull()), 'BMI'] = Diabetes['BMI'].median()



In [ ]:
#according to the data distribution :
# we will impute missing values with median , according to Diabetes and no diabetes data sets : 

df.loc[(df['Outcome'] == 0 ) & (df['Insulin'].isnull()), 'Insulin'] = No_diab['Insulin'].mean()
df.loc[(df['Outcome'] == 1 ) & (df['Insulin'].isnull()), 'Insulin'] = Diabetes['Insulin'].mean()
df.loc[(df['Outcome'] == 0 ) & (df['Glucose'].isnull()), 'Glucose'] = No_diab['Glucose'].mean()
df.loc[(df['Outcome'] == 1 ) & (df['Glucose'].isnull()), 'Glucose'] = Diabetes['Glucose'].mean()
df.loc[(df['Outcome'] == 0 ) & (df['SkinThickness'].isnull()), 'SkinThickness'] = No_diab['SkinThickness'].mean()
df.loc[(df['Outcome'] == 1 ) & (df['SkinThickness'].isnull()), 'SkinThickness'] = Diabetes['SkinThickness'].mean()
df.loc[(df['Outcome'] == 0 ) & (df['BloodPressure'].isnull()), 'BloodPressure'] = No_diab['BloodPressure'].mean()
df.loc[(df['Outcome'] == 1 ) & (df['BloodPressure'].isnull()), 'BloodPressure'] = Diabetes['BloodPressure'].mean()
df.loc[(df['Outcome'] == 0 ) & (df['BMI'].isnull()), 'BMI'] = No_diab['BMI'].mean()
df.loc[(df['Outcome'] == 1 ) & (df['BMI'].isnull()), 'BMI'] = Diabetes['BMI'].mean()



**Now let's look to our data after modification :**

In [ ]:
df.isnull().sum().plot(kind='bar')

In [ ]:
df.hist(figsize=(15,12));

In [ ]:
df.columns

In [ ]:
print(df.shape)

In [ ]:
#Spliting Data : 

x=df.iloc[:,:-1].values
y=df.iloc[:,-1].values

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.20, random_state = 0)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()

x_train_std = ss.fit_transform(x_train)
x_test_std = ss.transform(x_test)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

knn = KNeighborsClassifier()

param_grid = {'n_neighbors':[5,10,15,25,30,50]}

grid_knn = GridSearchCV(knn,param_grid,scoring='roc_auc',cv = 10,refit = True,n_jobs=-1)

In [ ]:
grid_knn.fit(x_train_std,y_train)
print("Best Score ==> ", grid_knn.best_score_)
print("Tuned Paramerers ==> ",grid_knn.best_params_)
print("Accuracy on Train set ==> ", grid_knn.score(x_train_std,y_train))
print("Accuracy on Test set ==> ", grid_knn.score(x_test_std,y_test))

In [ ]:
y_pred_K = grid_knn.best_estimator_.predict(x_test_std)
cnf_matrix = confusion_matrix(y_test, y_pred_K)
p = sns.heatmap(pd.DataFrame(cnf_matrix), annot=True, cmap="YlGnBu" ,fmt='g')
plt.title('Confusion matrix', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')

In [ ]:
print(classification_report(y_test, y_pred_K))

In [ ]:
from sklearn.svm import SVC

svc = SVC(probability=True)

param_grid = {'kernel':['rbf','linear'],'C':[0.01,0.1,1,0.001],'gamma':[0.1,0.01,0.2,0.4]}

grid_svc = GridSearchCV(svc,param_grid,cv = 10,refit = True,scoring='roc_auc',n_jobs=-1)

In [ ]:
y_pred_s=grid_svc.fit(x_train_std,y_train)
print("Best Score ==> ", grid_svc.best_score_)
print("Tuned Paramerers ==> ",grid_svc.best_params_)
print("Accuracy on Train set ==> ", grid_svc.score(x_train_std,y_train))
print("Accuracy on Test set ==> ", grid_svc.score(x_test_std,y_test))

In [ ]:
y_pred_s = grid_svc.best_estimator_.predict(x_test_std)
cnf_matrix = confusion_matrix(y_test, y_pred_s)
p = sns.heatmap(pd.DataFrame(cnf_matrix), annot=True, cmap="YlGnBu" ,fmt='g')
plt.title('Confusion matrix', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')

In [ ]:
print(classification_report(y_test, y_pred_s))

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(random_state=0)

param_grid ={
    'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
    'max_iter': list(range(100,800,100)),
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
}

grid_lr = GridSearchCV(lr,param_grid,scoring='accuracy',cv = 10,refit = True)

In [ ]:
y_red_l=grid_lr.fit(x_train_std,y_train)
print("Best Score ==> ", grid_lr.best_score_)
print("Tuned Paramerers ==> ",grid_lr.best_params_)
print("Accuracy on Train set ==> ", grid_lr.score(x_train_std,y_train))
print("Accuracy on Test set ==> ", grid_lr.score(x_test_std,y_test))

In [ ]:
y_red_l = grid_lr.best_estimator_.predict(x_test_std)
cnf_matrix = confusion_matrix(y_test, y_red_l)
p = sns.heatmap(pd.DataFrame(cnf_matrix), annot=True, cmap="YlGnBu" ,fmt='g')
plt.title('Confusion matrix', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')

In [ ]:
print(classification_report(y_test, y_red_l))

**Now let's try combining these results and see if we can do better :**

In [ ]:
from sklearn.ensemble import VotingClassifier

classifiers = [('knn',grid_knn),('tree',grid_lr),('svc',grid_svc)]

vtc = VotingClassifier(classifiers)

In [ ]:

vtc.fit(x_train_std,y_train)
y_pred_v=vtc.predict(x_test_std)
print("Accuracy on Train set ==> ", vtc.score(x_train_std,y_train))
print("Accuracy on Test set ==> ", vtc.score(x_test_std,y_test))

In [ ]:

print(classification_report(y_test, y_pred_v))